In [3]:
from facemesh.faceDetector import FaceDetector, cv2
from facemesh.utils import eye_aspect_ratio

import numpy as np
import time


def draw_polyline(img, l):
    cv2.polylines(img, np.array([l], dtype=np.int32), True, (0,255,0), 1, cv2.LINE_AA)

## Landmarks keypoints:
## https://github.com/tensorflow/tfjs-models/blob/838611c02f51159afdd77469ce67f0e26b7bbb23/face-landmarks-detection/src/mediapipe-facemesh/keypoints.ts
lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]

cap = cv2.VideoCapture(0)
pTime = 0
detector = FaceDetector()

# counters:
blinkCount = 0

while True:
    left_eye_lm = []
    right_eye_lm = []
    lipsUpperOuter_lm, lipsLowerOuter_lm, lipsUpperInner_lm, lipsLowerInner_lm = [],[],[],[]
    success, img = cap.read()
    if success:
        l = detector.facemeshing(img, draw = False)
        if l:
            # Get left and right eye
            for lm_id in LEFT_EYE:
                left_eye_lm.append(l[0][lm_id])
            
            for lm_id in RIGHT_EYE:
                right_eye_lm.append(l[0][lm_id])
            # Get lips: 
            for lm_id in lipsUpperOuter:
                lipsUpperOuter_lm.append(l[0][lm_id])
            
            for lm_id in lipsLowerOuter:
                lipsLowerOuter_lm.append(l[0][lm_id])
            for lm_id in lipsUpperInner:
                lipsUpperInner_lm.append(l[0][lm_id])
            
            for lm_id in lipsLowerInner:
                lipsLowerInner_lm.append(l[0][lm_id])
            
            # getting leftEAR from eye_aspect_ratio.
            # eye_aspect_ratio computes distances betwen the two sets of vertical eye landmarks coordinates.

            draw_polyline(img,left_eye_lm )
            draw_polyline(img,right_eye_lm )
            draw_polyline(img,lipsUpperOuter_lm + lipsLowerOuter_lm[::-1])
            draw_polyline(img,lipsUpperInner_lm + lipsLowerInner_lm[::-1])
        else:
            pass
    cTime = time.time()
    fps = 1 / (cTime-pTime)
    pTime = cTime
    
    cv2.putText(img, f'FPS: {int(fps)}', (20,50), cv2.FONT_HERSHEY_PLAIN,3, (0,255, 255), 2)
    cv2.imshow('face',img)
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        cap.release() 
        break  
cv2.destroyAllWindows()

In [1]:
from modules.faceDetector import FaceDetector, cv2
from modules.utils import eye_aspect_ratio, mouth_aspect_ratio, rec_to_roi_box, crop_img
from modules.facepose import Facepose
from imutils import face_utils
from PIL import Image
import numpy as np
import math
import time
import dlib


def draw_polyline(img, l):
    cv2.polylines(img, np.array([l], dtype=np.int32), True, (0,255,0), 1, cv2.LINE_AA)

def showFPS(img, pTime):
    cTime = time.time()
    fps = 1 / (cTime-pTime)
    pTime = cTime
    cv2.putText(img, f'FPS: {int(fps)}', (20,50), cv2.FONT_HERSHEY_PLAIN, 3, (0,255, 255), 2)
    return pTime, cTime

def averageFPS(img, counter, pTime, prev_fps, interval = 1 ):
    counter +=1
    cTime = time.time()
    if cTime-pTime > interval:
        fps = int(counter/(cTime-pTime))
        counter = 0
        pTime = cTime
    else:
        fps = prev_fps
    #cv2.putText(img, f'FPS: {int(fps)}', (80,50), cv2.FONT_HERSHEY_PLAIN, 3, (0,255, 255), 2)
    return fps, pTime, counter

def euclaidean_distance(p1, p2):
    x,y = p1
    x1, y1 = p2
    distance = math.sqrt((x1-x)**2 + (y1-1)**2)
    return distance
    
def blinking_ratio(img, landmarks, right_indices, left_indices, draw = False):
    # right eye
    # horizontal line
    rh_right = landmarks[right_indices[0]]
    rh_left = landmarks[right_indices[8]]
    # vertical line
    rv_top = landmarks[right_indices[12]]
    rv_bottom = landmarks[right_indices[4]]

    if draw:
        cv2.line(img, rh_right,rh_left, (255,128,0), 2)
        cv2.line(img, rv_top,rv_bottom, (0,0,0), 2)

    # left eye
    # horizontal line
    lh_right = landmarks[right_indices[0]]
    lh_left = landmarks[right_indices[8]]
    # vertical line
    lv_top = landmarks[right_indices[12]]
    lv_bottom = landmarks[right_indices[4]]
    pass


## Landmarks keypoints:
## https://github.com/tensorflow/tfjs-models/blob/838611c02f51159afdd77469ce67f0e26b7bbb23/face-landmarks-detection/src/mediapipe-facemesh/keypoints.ts
lipsUpperOuter = [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291]
lipsLowerOuter = [146, 91, 181, 84, 17, 314, 405, 321, 375, 291]
lipsUpperInner = [78, 191, 80, 81, 82, 13, 312, 311, 310, 415, 308]
lipsLowerInner = [78, 95, 88, 178, 87, 14, 317, 402, 318, 324, 308]
LEFT_EYE =[ 362, 382, 381, 380, 374, 373, 390, 249, 263, 466, 388, 387, 386, 385,384, 398 ]
RIGHT_EYE=[ 33, 7, 163, 144, 145, 153, 154, 155, 133, 173, 157, 158, 159, 160, 161 , 246 ] 
LEFT_IRIS = [474, 475, 476, 477]
RIGHT_IRIS = [469, 470, 471, 472]


#hopenet_dir = '../../model/hopenet_robust_alpha1.pkl'
model_dir = "../../model/shape_predictor_68_face_landmarks.dat"
faceDetector = FaceDetector()
#facepose = Facepose(hopenet_dir)
predictor = dlib.shape_predictor(model_dir)
cap = cv2.VideoCapture(0)

# counters:
blinkCount = 0
yawnCount = 0
fps, pTime, frame_counter = 0,0,0

lostFocusCount = 0
lostFocusDuration = 0
focusTimer = None
# states
yawning = False
eyeClosed = False
lostFocus = False

while True:
    left_eye_lm = []
    right_eye_lm = []
    lipsUpperOuter_lm, lipsLowerOuter_lm, lipsUpperInner_lm, lipsLowerInner_lm = [],[],[],[]
    success, frame = cap.read()
    
    if success: ## si captura imagen.
        frame_gray = cv2.cvtColor(frame.copy(), cv2.COLOR_BGR2GRAY)
        faces = faceDetector.findFaces(frame, d_square=False)
        faceDetector.head_tilting(frame)

        #print(bb[0][0]) ## one face (4 coordinates) and rate.
        if not len(faces) == 0: 
            for rostro in faces[0][0]:
                x, y, w, h = faces[0][0] #first point, bridging point
                x1, y1 = x+w, y+h #bottom right point
                cv2.rectangle(frame,(x, y),(x1, y1), (0,255,0),1 )
                

                leftEye = LEFT_EYE
                rightEye = RIGHT_EYE
                
                
                blinking_ratio(frame, rightEye, leftEye)
                
                '''dlib too slow

                rect = dlib.rectangle(left= x, top= y, right= x1, bottom= y1 ) # generar rectángulo
                
                
                
                
                
                
                shape = predictor(frame_gray, rect) 
                shape = face_utils.shape_to_np(shape)

                leftEye = shape[36:42]
                rightEye = shape[42:48]
                leftEAR = eye_aspect_ratio(leftEye)
                rightEAR = eye_aspect_ratio(rightEye)
                ear = (leftEAR + rightEAR) / 2.0
                mar = mouth_aspect_ratio(shape[60:69])

                if ear < 0.15:
                    eyeClosed = True
                if ear > 0.15 and eyeClosed:
                    blinkCount += 1
                    eyeClosed = False

                if mar > 0.4:
                    yawning = True
                if mar < 0.2 and yawning:
                    yawnCount += 1
                    yawning = False

                roi_box, center_x, center_y = rec_to_roi_box(rect)
                roi_img = crop_img(frame, roi_box)
                img = Image.fromarray(roi_img)
                '''
                
        else:
            pass

    # FPS 
    fps, pTime, frame_counter = averageFPS(frame,frame_counter, pTime, fps)
    #pTime, cTime = showFPS(frame, pTime)
    cv2.putText(frame, f'FPS: {int(fps)}', (80,50), cv2.FONT_HERSHEY_PLAIN, 3, (0,255, 255), 2)

    cv2.imshow('face',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release() 
        break  
cv2.destroyAllWindows()

ValueError: Input vector should be 1-D.

In [1]:
from modules.faceDetector import FaceDetector, cv2
from modules.units.mouthUnit import MouthUnit
import time

def averageFPS(img, counter, pTime, prev_fps, interval = 1 ):
    counter +=1
    cTime = time.time()
    if cTime-pTime > interval:
        fps = int(counter/(cTime-pTime))
        counter = 0
        pTime = cTime
    else:
        fps = prev_fps
    cv2.putText(img, f'FPS: {int(fps)}', (80,50), cv2.FONT_HERSHEY_PLAIN, 3, (0,255, 255), 2)
    return fps, pTime, counter

facer = FaceDetector()
mouth = MouthUnit()

cap = cv2.VideoCapture(0)
pTime = 0
frame_counter= 0
fps = 0

while True:
    ret, frame = cap.read()
    if ret:
        frame_shape = frame.shape[:2]
        #facer.findFaces(frame, d_square=True)
        results = facer.get_results(frame)
        #facer.facemeshing(results,frame, draw=False)
        mesh_list = facer.landmarksDetection(frame, results, frame_shape, draw=False)
        mouth.mouth_aspect_ratio(frame, mesh_list, draw=True)

        # FPS     
    #pTime, cTime = showFPS(frame, pTime)
    fps, pTime, frame_counter = averageFPS(frame, frame_counter, pTime, fps)

    #cv2.putText(frame, f'FPS: {int(fps)}', (20,50), cv2.FONT_HERSHEY_PLAIN, 3, (0,255, 255), 2)
    cv2.imshow('face',frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release() 
        break  
cv2.destroyAllWindows()


    